In [50]:
import pandas as pd
from tabulate import tabulate
from scipy.stats import ttest_ind
import warnings
pd.set_option('display.max_columns', None)

In [51]:
play2024_df = pd.read_csv("data/play_by_play_2024.csv")
injury2024_df = pd.read_csv("data/injuries_2024.csv")

play2023_df = pd.read_csv("data/play_by_play_2023.csv")
injury2023_df = pd.read_csv("data/injuries_2023.csv")

play2022_df = pd.read_csv("data/play_by_play_2022.csv")
injury2022_df = pd.read_csv("data/injuries_2022.csv")

play2021_df = pd.read_csv("data/play_by_play_2021.csv")
injury2021_df = pd.read_csv("data/injuries_2021.csv")



In [52]:
def first_last_a(name):
    parts = name.split(" ", 1)
    return f"{parts[0][0]}.{parts[1]}"

In [53]:
def first_last_b(name):
    parts = name.split(" ", 1)
    return f"{parts[0][0:2]}.{parts[1]}"

In [54]:
warnings.filterwarnings('ignore')

def merge_play_injury_dfs(play_df, injury_df):

    # preprocessing
    injury_df["date"] = pd.to_datetime(injury_df['date_modified'])
    play_df["date"] = pd.to_datetime(play_df['game_date'])
    injury_df['date'] = injury_df['date'].dt.tz_localize(None)
    play_df['date'] = play_df['date'].dt.tz_localize(None)

    # filtering
    plays_with_injuries = play_df[play_df['desc'].str.contains("was injured", na=False)]
    pattern = r'(\w+\.(?:\w|-|\.|\')+(?: \w+)*) was injured'
    # Extract the injured player's name from the desc column
    injured_players = plays_with_injuries.loc[:, "desc"].str.extract(pattern)

    # concatenation
    plays_with_injuries = pd.concat([plays_with_injuries, injured_players], axis=1)
    plays_with_injuries.rename(columns={0: "injured_player"}, inplace=True)
    plays_with_injuries = plays_with_injuries.reset_index(drop=True)
    
    # merging
    injuries = []
    for (week, team), group_injury_df in injury_df.groupby(['week', 'team']):
        group_play_df = plays_with_injuries[(plays_with_injuries['week'] == week) & ((plays_with_injuries['home_team'] == team) | (plays_with_injuries['away_team'] == team))]

        #group_injury_df = group_injury_df[group_injury_df.date >= group_play_df.date.max()]

        group_injury_df["first_type"] = group_injury_df['full_name'].apply(first_last_a)
        group_injury_df["second_type"] = group_injury_df['full_name'].apply(first_last_b)

        x = pd.merge(group_play_df, group_injury_df, left_on="injured_player", right_on="first_type", how="inner")
        y = pd.merge(group_play_df, group_injury_df, left_on="injured_player", right_on="second_type", how="inner")

        injuries.append(pd.concat([x, y], axis = 0, ignore_index=True))

    plays_with_injuries_and_injury_record = (pd.concat(injuries, axis=0, ignore_index=True)).drop(columns=["first_type", "second_type"])
    plays_with_injuries_and_injury_record = plays_with_injuries_and_injury_record.sort_values('play_id', ascending=False).drop_duplicates(subset=['week_x', 'full_name', "team"], keep='first')

    return plays_with_injuries, plays_with_injuries_and_injury_record
#returns (plays where injuries occurred, plays were injuries occurred and missed time)



In [55]:
def populate_cols_in_play_df(play_df, plays_with_injuries, plays_with_injuries_and_injury_record):
    columns_to_check = ['play_id', 'game_id']
    play_df["was_injured"] = 0
    play_df["missed_time"] = 0
    play_df.loc[play_df[columns_to_check].apply(tuple, 1).isin(plays_with_injuries[columns_to_check].apply(tuple, 1)), 'was_injured'] = 1
    play_df.loc[play_df[columns_to_check].apply(tuple, 1).isin(plays_with_injuries_and_injury_record[columns_to_check].apply(tuple, 1)), 'missed_time'] = 1

    return play_df

In [56]:
plays_with_injuries_2024, plays_with_injuries_and_injury_record_2024 = merge_play_injury_dfs(play2024_df, injury2024_df)
plays_with_injuries_2023, plays_with_injuries_and_injury_record_2023 = merge_play_injury_dfs(play2023_df, injury2023_df)
plays_with_injuries_2022, plays_with_injuries_and_injury_record_2022 = merge_play_injury_dfs(play2022_df, injury2022_df)
plays_with_injuries_2021, plays_with_injuries_and_injury_record_2021 = merge_play_injury_dfs(play2021_df, injury2021_df)


In [57]:
# Print number of rows for each returned table
print("Rows in plays_with_injuries_2024:", len(plays_with_injuries_2024))
print("Rows in plays_with_injuries_and_injury_record_2024:", len(plays_with_injuries_and_injury_record_2024))

print("Rows in plays_with_injuries_2023:", len(plays_with_injuries_2023))
print("Rows in plays_with_injuries_and_injury_record_2023:", len(plays_with_injuries_and_injury_record_2023))

print("Rows in plays_with_injuries_2022:", len(plays_with_injuries_2022))
print("Rows in plays_with_injuries_and_injury_record_2022:", len(plays_with_injuries_and_injury_record_2022))

print("Rows in plays_with_injuries_2021:", len(plays_with_injuries_2021))
print("Rows in plays_with_injuries_and_injury_record_2021:", len(plays_with_injuries_and_injury_record_2021))


Rows in plays_with_injuries_2024: 457
Rows in plays_with_injuries_and_injury_record_2024: 231
Rows in plays_with_injuries_2023: 989
Rows in plays_with_injuries_and_injury_record_2023: 188
Rows in plays_with_injuries_2022: 902
Rows in plays_with_injuries_and_injury_record_2022: 206
Rows in plays_with_injuries_2021: 941
Rows in plays_with_injuries_and_injury_record_2021: 190


In [58]:
play2024_df = populate_cols_in_play_df(play2024_df, plays_with_injuries_2024, plays_with_injuries_and_injury_record_2024)
play2023_df = populate_cols_in_play_df(play2024_df, plays_with_injuries_2023, plays_with_injuries_and_injury_record_2023)
play2022_df = populate_cols_in_play_df(play2024_df, plays_with_injuries_2022, plays_with_injuries_and_injury_record_2022)
play2021_df = populate_cols_in_play_df(play2024_df, plays_with_injuries_2021, plays_with_injuries_and_injury_record_2021)